In [2]:
import os
import pymssql

import pandas as pd

from dotenv import load_dotenv

In [1]:
def get_connection():
    load_dotenv()
    return pymssql.connect(
        server=os.getenv("SQL_SERVER_HOST"),
        user=os.getenv("SQL_SERVER_USER"),
        password=os.getenv("SQL_SERVER_PASS"),
        port=os.getenv("SQL_SERVER_PORT"),
        database='processo'
    )

In [4]:
with open('consultas/processos_tramitacao_cip.sql', 'r') as f:
    tramitacao_sql = f.read()

In [5]:
df_tramitacao = pd.read_sql(tramitacao_sql, get_connection())

/tmp/ipykernel_12561/2479143534.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tramitacao = pd.read_sql(tramitacao_sql, get_connection())


In [42]:
def get_recebido_dam(row):
    with open('consultas/lotes_tramitacao_cip.sql', 'r') as f:
        lotes = f.read()
    lotes = pd.read_sql(lotes.format(numero_processo=row.numero_processo, ano_processo=row.ano_processo), get_connection())
    lotes.destino = lotes.destino.apply(lambda x: x.strip())
    return lotes[lotes.destino == 'DAM'].recebido_em.values[0]

In [45]:
df_tramitacao['recebido_dam'] = df_tramitacao.apply(get_recebido_dam, axis=1)

/tmp/ipykernel_12561/2252254553.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lotes = pd.read_sql(lotes.format(numero_processo=row.numero_processo, ano_processo=row.ano_processo), get_connection())


In [48]:
rel_tramitacao = df_tramitacao[['numero_processo', 'ano_processo', 'recebido_dam']]

In [52]:
rel_tramitacao['recebido_dam'] = pd.to_datetime(rel_tramitacao['recebido_dam']).dt.strftime('%d/%m/%Y')

/tmp/ipykernel_12561/894465215.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_tramitacao['recebido_dam'] = pd.to_datetime(rel_tramitacao['recebido_dam']).dt.strftime('%d/%m/%Y')


In [55]:
rel_tramitacao.to_excel('saidas/tramitacao/tramitacao_dam.xlsx', index=False)